<h2>Loading Data</h2>

The csvloader module supports loading information from the provided csv files. The script below prints basic information about the data.


In [1]:
from core.loaders.csvloader import csvloader;

path_test_data = './data/test.csv'
path_train_data = './data/train.csv'
path_ideal_data = './data/ideal.csv'

def load_data(path, label):
    data_importer = csvloader.CSVLoader(path, label)
    data_importer.read_csv()
    return data_importer

def print_csv_info(path, label):
    loader = load_data(path, label)
    loader.print_csv_info()

print_csv_info(path_test_data, 'test.csv')
print_csv_info(path_train_data, 'train.csv')
print_csv_info(path_ideal_data, 'ideal.csv')



file information for test.csv: {"size": 200, "shape": [100, 2], "dimensions": 2, "type": "float64"}
file information for train.csv: {"size": 2000, "shape": [400, 5], "dimensions": 2, "type": "float64"}
file information for ideal.csv: {"size": 20400, "shape": [400, 51], "dimensions": 2, "type": "float64"}


<h1>Cleaning Data</h1> 
The csvcleanser module cleans and prepares the data for analysis. It removes any NA entries and prepares a sorted data set by the first column.
Values that are 2* standard deviations from the mean are considered outliers.



In [2]:
from core.cleanser.csvcleanser import csvcleanser;

def print_cleansing_info(path, label):
    loader = load_data(path, label)
    data_cleanser = csvcleanser.CSVCleanser(loader.csv_data, loader.label)
    data_cleanser.print_cleansing_info()

print_cleansing_info(path_test_data, 'test.csv')
print_cleansing_info(path_train_data, 'train.csv')
print_cleansing_info(path_ideal_data, 'ideal.csv')


cleaning file information for test.csv: {"columns_with_na": [], "num_columns": 2, "sorted_by_index": 0, "removed_duplicated": 0} with 11 Outliers
cleaning file information for train.csv: {"columns_with_na": [], "num_columns": 5, "sorted_by_index": 0, "removed_duplicated": 0} with 20 Outliers
cleaning file information for ideal.csv: {"columns_with_na": [], "num_columns": 51, "sorted_by_index": 0, "removed_duplicated": 0} with 0 Outliers
